In [8]:
# from flask_other.app import app
from flask import Flask
import os,sys
# sys.path.append('/home/kai/data/wei/Chatbot1.0/')
sys.path.append('/home/kai/data/Chatbot1.0/')
app=Flask("demo")
from flask import  render_template,request
from app.mong_database import MongoManager
import time
import re
mongo_db=MongoManager(server_ip='chatbotdb')

import urllib.request
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import re

def get_data(sentence):
    #url中含有中文时要单独处理
    req = urllib.request.Request(
        'http://drea.cc/api/chat.php?msg={}&uid=drea_bbs_chat'.format(quote(sentence)))
    req.add_header('Content-type', 'text/xml; charset="gbk"')
    response = urllib.request.urlopen(req)
    buff = response.read()
    the_page = buff.decode('gbk')
    print(type(the_page),eval(the_page)['reply'])
    response.close()
    return eval(the_page)['reply']

def get_data(sentence):
   
    return sentence



@app.route('/')
@app.route('/index')
def index0():
    return render_template(
                           # "index0.html",
                           "index.html",
                           )
@app.route("/predict", methods= ["POST"])
def background_process():
    if request.method == 'POST':
        try:
            query = request.form.get('query')#前端查询的内容
            if query:

                    print('query',query)
                    # time.sleep(5)
                    result = get_data(query)
                    print('result',result)
                    mongo_db.save_query(query, str(result))
                    print('saving to mongo db successfully!')
                    return str(result)
                    

            else:

                    return str('请输入查询内容')


        except Exception as e:
            print(e)

            if 'duplicate' in str(e):
                e_str = e.details['errmsg']
                dup_id=re.search('\{ : "(.*)" \}',e_str).group(1)
                print('重复查询同一句话,存储时使用相同的_id_',dup_id)
                mongo_db.update_dup_query( dup_id, str(result))
                return str(result)

            else:
                print(e)
                print('有问题，MM出故障啦')
                return str('MM出故障啦')

        # finally:
        #     # print(e)
        #     print('有问题，MM出故障啦。。')
        #     return str('MM出故障啦。。')

    else:
        return 'ok'

# @app.route('/dataFromAjax_post',methods=['POST','GET'])
# def dataFromAjax_post():
#     if request.method == 'POST':
#
#         try:
#             # query = request.form.get('mydata')  # 前端查询的内容
#             query = request.form['mydata']  # 前端查询的内容
#             if query:
#
#                 print('query', query)
#                 # time.sleep(5)
#                 result = '欢迎光临'
#                 print('result', result)
#                 # mongo_db.save_query(query, str(result))
#                 return str(result)
#             else:
#
#                 return str('请输入查询内容')
#         except Exception as e:
#             print('exception', e)
#             return str('有问题')
#     else:
#         return 'ok'





chatbotdb


In [ ]:
app.debug = False
app.run(host='0.0.0.0',port=8889)    #这样用来监听所有的ip，团队调试用


 * Running on http://0.0.0.0:8889/ (Press CTRL+C to quit)
172.18.0.1 - - [07/Jun/2018 22:23:27] "POST /dataFromAjax_post HTTP/1.1" 404 -
172.18.0.1 - - [07/Jun/2018 22:23:27] "POST /predict HTTP/1.1" 200 -


query 你好
result 你好
saving to mongo db successfully!


In [4]:
import sys,os
sys.path.append(os.path.dirname(__file__),'../../')

NameError: name '__file__' is not defined